In [3]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
import os
os.mkdir('images')
from fastbook import *

from fastai.vision.all import *
from fastai.vision.widgets import *

ModuleNotFoundError: ignored

In [ ]:
key = os.environ.get('AZURE_SEARCH_KEY', '4becca12d23f4739b3d0155eee322bbc')

results = search_images_bing(key, 'nike shoes')
ims = results.attrgot('content_url')
len(ims)

ims = ['https://static.nike.com/a/images/t_PDP_1280_v1/f_auto,q_auto:eco/c6081451-e5e6-44a2-a4f4-21558717782f/classic-cortez-shoe-z2JGFJ.jpg']

dest = 'images/nike.jpg'
download_url(ims[0], dest)

im = Image.open(dest)
im.to_thumb(128,128)

shoes_types = 'nike','reebok','adidas'
path = Path('bears')

if not path.exists():
    path.mkdir()
    for o in shoes_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o} shoes')
        download_images(dest, urls=results.attrgot('contentUrl'))
        
fns = get_image_files(path)
fns
failed = verify_images(fns)
failed.map(Path.unlink);

shoes = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

dls = shoes.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

shoes = shoes.new(item_tfms=Resize(128, ResizeMethod.Squish))
dls = shoes.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

shoes = shoes.new(item_tfms=Resize(128, ResizeMethod.Pad, pad_mode='zeros'))
dls = shoes.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

shoes = shoes.new(item_tfms=RandomResizedCrop(128, min_scale=0.3))
dls = shoes.dataloaders(path)
dls.train.show_batch(max_n=4, nrows=1, unique=True)

shoes = shoes.new(item_tfms=Resize(128), batch_tfms=aug_transforms(mult=2))
dls = shoes.dataloaders(path)
dls.train.show_batch(max_n=8, nrows=2, unique=True)

shoes = shoes.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = shoes.dataloaders(path)

learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

interp.plot_top_losses(5, nrows=1)

cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
path = Path()
learn_inf = load_learner(path/'export.pkl')
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')

In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'


In [ ]:
btn_run.on_click(on_click_classify)

In [ ]:
VBox([widgets.Label('Select shoes!'), btn_upload, btn_run, out_pl, lbl_pred])